In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [2]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 창 없이 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(options=options)

In [3]:
# Navigate to the website
url = "https://www.construction-europe.com/search?keyword=Komatsu"
driver.get(url)
time.sleep(2)

In [4]:
# Click the cookie button
driver.find_element(By.CSS_SELECTOR, '#cookieConsent > div > div > div > div.col-12.col-md-auto.text-center.py-2.align-self-center > button').click()

In [5]:
req = urllib.request.Request(url)
sourcecode = urllib.request.urlopen(url).read()
soup = BeautifulSoup(sourcecode, "html.parser")

In [6]:
# Click the options
brand = driver.find_element(By.ID, 'BrandFilter_3')
driver.execute_script("arguments[0].click();", brand)

type = driver.find_element(By.ID, 'TypeFilter_1')
driver.execute_script("arguments[0].click();", type)

category = driver.find_element(By.ID, 'CategoryFilter_1')
driver.execute_script("arguments[0].click();", category)

# Apply the selected options
driver.find_element(By.CSS_SELECTOR, '#filterOverlay > div > div > div > button.btn.btn-primary.btn-lg.btn-ApplyFilter').click()

In [7]:
# Scrape article details
full_address = []

for i in range(3):
    articles = soup.find_all("div", class_="khl-article-block-headline font-weight-bold h4")
    for article in articles:
        address = article.find("a")["href"]
        if address != 'articleurl' and address not in full_address:  # 'articleurl'이 아니고 중복되지 않은 경우에만 추가
            full_address.append(address)
        print(full_address)
        
    next_page_button = driver.find_element(By.CSS_SELECTOR, '#pagination > li.page-item.next > a').click()
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, "html.parser")

full_address

['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article']
['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article', 'https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article']
['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article', 'https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article', 'https://www.international-construction.com/news/why-komatsu-has-started-factory-filling-machines-with-hvo/8029103.article']
['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article', 'https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article', 'https://www.international-construction.com/news/why-komatsu-has-sta

['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article',
 'https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article',
 'https://www.international-construction.com/news/why-komatsu-has-started-factory-filling-machines-with-hvo/8029103.article',
 'https://www.dieselprogress.com/news/komatsu-toyota-to-develop-autonomous-light-vehicles/8029091.article',
 'https://www.construction-europe.com/news/komatsu-launches-mid-sized-hydrogen-fuel-cell-excavator-concept/8029004.article',
 'https://www.construction-europe.com/news/digital-construction-site-steals-show-on-samoter-s-first-day/8028738.article',
 'https://www.DemolitionandRecycling.media/news/komatsu-to-switch-to-hvo-as-factory-fill-fuel/8028428.article',
 'https://www.international-construction.com/news/what-can-construction-learn-from-japan-/8027856.article',
 'https://www.international-construction.com/news/komatsu-s-tech-s

In [8]:
article_data = []
for article_link in full_address:
    driver.get(article_link)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    release_date = ""
    title = ""
    content = ""
   
    # Scrape release date
    release_date_elem = soup.find("span", class_="PubDate")
    release_date = release_date_elem.text.strip() if release_date_elem else ""
            
    # Scrape news title
    title_elem = soup.find("h1", class_="khl-article-page-title")
    title = title_elem.text.strip() if title_elem else ""

    # Scrape article content
    first = soup.find("div", class_="col-12 khl-article-page-standfirst").text
    content_elem = soup.find("div", class_="col-12 khl-article-page-storybody")
    paragraphs = content_elem.find_all("p") if content_elem else []
    content = "\n".join([p.text.strip() for p in paragraphs])
    content_elem = soup.find("div", class_="col-12 khl-article-page-storybody")
    paragraphs = content_elem.find_all("p") if content_elem else []
    content = "\n".join([p.text.strip() for p in paragraphs])

    # Check if content is empty
    if not content:
        continue

    # Append article data to the list
    article_data.append({"Date": release_date, "Title": title, "Content": content})
driver.quit()

In [9]:
# Create DataFrame from article data
df = pd.DataFrame(article_data)
df

,Date,Title,Content
0,30 May 2023,World’s top ten construction OEMs revealed,"The latest Yellow Table, International Constru..."
1,26 May 2023,Heavy wheeled loaders: The workhorse of the co...,Heavy wheeled loaders have a home in almost ev...
2,24 May 2023,Why Komatsu has started factory-filling machin...,\n\nKomatsu’s customers are still concerned ab...
3,18 May 2023,"Komatsu, Toyota to develop autonomous light ve...",\n\nKomatsu Ltd. and Toyota Motor Corp. have l...
4,16 May 2023,Komatsu launches mid-sized hydrogen fuel-cell ...,\nThe manufacturer said the machine’s developm...
5,03 May 2023,Digital construction site steals show on Samot...,"\n\n\n\n\nMachine control, remote control, aut..."
6,,Komatsu to switch to HVO as factory-fill fuel,"The move, which is line with Komatsu’s long-te..."
7,11 April 2023,What can construction learn from Japan?,Japan’s approach to safety and quality in cons...
8,22 March 2023,Komatsu’s tech solutions to boost construction...,"At ConExpo in Las Vegas, US, Komatsu, one of t..."
9,10 March 2023,The sky’s the limit: the latest in drone techn...,"Drones, also known as unmanned aerial vehicles..."


In [10]:
# Save DataFrame to CSV
df.to_csv("komatsu_article_final.csv", index=False)